# Utils

## Connect to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

## Label the files with the correct meaning of a sign for LIBRAS

In [ ]:
!pip install pympi-ling

In [ ]:
import pandas as pd
import pympi.Elan as Elan
import math

video_str = 'FLN_G1_D4_VOCAB_2Alimentos_VIDEO2'
eaf_str = 'FLN_G1_D4_VOCAB_2Alimentos_VIDEO1234'
fps = 30
fpms = fps / 1000
df = pd.read_excel('/content/gdrive/My Drive/your_directory/LIBRAS Data/'+video_str+'_all_output_07102022.xlsx', index_col=0)
elan_object = Elan.Eaf(file_path = '/content/gdrive/My Drive/your_directory/LIBRAS Data/'+eaf_str+'.eaf')
xml_rhlabel = "1SinaisD"
#xml_rhlabel = "RH-IDgloss"
xml_lhlabel = "1SinaisE"
#xml_lhlabel = "LH-IDgloss"
# iterate through each row and select 
#df.drop('class',  axis='columns', inplace=True)
#df.drop('video', axis='columns', inplace=True)
df["label"] = None

for i in range(len(df)) :
  frame = df["frame"][i]
  #print(f'Frame {frame}')
  for annotation in elan_object.get_annotation_data_for_tier(xml_rhlabel):
    #print(annotation)
    # We are interested in the utterance
    utterance = annotation[2].strip().lower()
    start_time = int(annotation[0])
    end_time = int(annotation[1])
    start_frame = math.ceil(start_time * fpms)
    end_frame = math.ceil(end_time * fpms)
    if frame >= start_frame and frame <= end_frame:
      df["label"][i] = utterance
      break
  if df["label"][i] == None:
    for annotation in elan_object.get_annotation_data_for_tier(xml_lhlabel):
    #print(annotation)
    # We are interested in the utterance
      utterance = annotation[2].strip().lower()
      start_time = int(annotation[0])
      end_time = int(annotation[1])
      start_frame = math.ceil(start_time * fpms)
      #df["start_frame"][i] = str(start_time) + "->" +  str(start_frame)
      end_frame = math.ceil(end_time * fpms)
      #df["end_frame"][i] = str(end_time) + "->" +  str(end_frame)
      if frame >= start_frame and frame <= end_frame:
        df["label"][i] = utterance
        break
  if df["label"][i] == None:
    df["label"][i] = 'blank_transition'

df.drop('frame',
  axis='columns', inplace=True)
}df.to_excel("/content/gdrive/My Drive/your_directory/LIBRAS Data/"+video_str+"_fortraining_output_07102022.xlsx")  

# Merge different features in one file

In [ ]:
import pandas as pd

video_str = '01067'
#df = pd.read_excel('/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Otoño 2022/LIBRAS Data/'+ video_str +'_faceposture_output_07102022_1aug.xlsx', index_col=0)
#df_gaze = pd.read_excel('/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Otoño 2022/LIBRAS Data/'+ video_str +'_gazehead_output_17102022.xlsx', index_col=0)
#df_speed = pd.read_excel('/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Otoño 2022/LIBRAS Data/'+ video_str +'_hands_output_17102022.xlsx', index_col=0)
df = pd.read_excel('/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Primavera 2023/WLASL/Data/'+ video_str +'_faceposture_output_25012023_1aug.xlsx', index_col=0)
df_gaze = pd.read_excel('/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Primavera 2023/WLASL/Data/'+ video_str +'_gazehead_output_25012023.xlsx', index_col=0)
df_speed = pd.read_excel('/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Primavera 2023/WLASL/Data/'+ video_str +'_hands_output_25012023.xlsx', index_col=0)
df_speed = df_speed.drop([0], axis=1)
column_names_df = df.columns.values.tolist()
last_column_name = column_names_df[-1]
indexes_gaze = []
for index in range(len(df_gaze.columns.values.tolist())):
  df[last_column_name+index+1] = 0
  indexes_gaze.append(last_column_name+index+1)
column_names_df = df.columns.values.tolist()
last_column_name = column_names_df[-1]
indexes_speed = []
for index in range(len(df_speed.columns.values.tolist())):
  df[last_column_name+index+1] = 0
  indexes_speed.append(last_column_name+index+1)
#print(indexes_gaze)
#print(indexes_speed)
#df.head()
count, count_aug = 0, 1
INCREASE_COUNT = False
count_total = 0
prev_frame = df.iloc[count_total, 0]
for index, row in df.iterrows():
    #if count > 26:
    #  break
    #print(count_total)
    #print(count)
    frame = df.iloc[count_total, 0]
    if prev_frame != frame:
      prev_frame = frame
      count += 1
    df.iloc[count_total, indexes_gaze] = df_gaze.iloc[count].tolist()
    count_total += 1
count, count_aug = 0, 1
INCREASE_COUNT = False
count_total = 0
prev_frame = df.iloc[count_total, 0]
for index, row in df.iterrows():
    #if count > 537:
    #  break
    #print(count)
    frame = df.iloc[count_total, 0]
    if prev_frame != frame:
      prev_frame = frame
      count += 1
    df.iloc[count_total, indexes_speed] = df_speed.iloc[count].tolist()
    count_total += 1

df.to_excel("/content/gdrive/My Drive/Doctorado/Programa Tesis/Avances Primavera 2023/WLASL/Data/"+ video_str +"_all_output_25012023.xlsx")
#X = df.drop("label", axis=1)

## Generate CSV files with correct format for Transformer

In [ ]:
!pip install pympi-ling

In [ ]:
import pandas as pd
import pympi.Elan as Elan
import math
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA

video_str = 'FLN_G1_D1_VOCAB_1Família_VIDEO3'
eaf_str = 'FLN_G1_D1_VOCAB_1Família_VIDEO123'
fps = 30
fpms = fps / 1000

df = pd.read_excel('/content/gdrive/My Drive/your_directory/LIBRAS Data/'+video_str+'_all_output_09092022.xlsx', index_col=0)
elan_object = Elan.Eaf(file_path = '/content/gdrive/My Drive/your_directory/LIBRAS Data/'+eaf_str+'.eaf')
xml_rhlabel = "2Sinais D"
#xml_rhlabel = "RH-IDgloss"
xml_lhlabel = "2Sinais E"
#xml_lhlabel = "LH-IDgloss"

#ss = StandardScaler()
ss = MinMaxScaler()
X = df.drop("frame", axis=1) # Create a data with all columns except labels
X_scaled = ss.fit_transform(X)
pca = PCA(0.95)
pca.fit(X_scaled)
X_scaled = pca.transform(X_scaled)
print(df.shape)
print(X_scaled.shape)

features_matrix_orig = []
features_matrix_aug = []
columns = []
for i in range(X_scaled.shape[1]):
  features_matrix_orig.append([])
  features_matrix_aug.append([])
  columns.append(i)
class_prev = ""
class_current = ""
#create empty dataframe
df_transformer = pd.DataFrame(columns=columns)
# iterate through each row and select 
for i in range(0, len(df), 2) :
  frame = df["frame"][i]
  #print(f'Frame {frame}')
  for annotation in elan_object.get_annotation_data_for_tier(xml_rhlabel):
    #print(annotation)
    # We are interested in the utterance
    utterance = annotation[2].strip().lower()
    start_time = int(annotation[0])
    end_time = int(annotation[1])
    start_frame = math.ceil(start_time * fpms)
    end_frame = math.ceil(end_time * fpms)
    if frame >= start_frame and frame <= end_frame:
      class_current = utterance
      break
  if class_current == "":
    for annotation in elan_object.get_annotation_data_for_tier(xml_lhlabel):
    #print(annotation)
    # We are interested in the utterance
      utterance = annotation[2].strip().lower()
      start_time = int(annotation[0])
      end_time = int(annotation[1])
      start_frame = math.ceil(start_time * fpms)
      #df["start_frame"][i] = str(start_time) + "->" +  str(start_frame)
      end_frame = math.ceil(end_time * fpms)
      #df["end_frame"][i] = str(end_time) + "->" +  str(end_frame)
      if frame >= start_frame and frame <= end_frame:
        class_current = utterance
        break
  #print(class_current)
  if class_current == "":
    class_current = 'blank_transition'
  
  if class_prev == "":
    class_prev = class_current
  #print(class_prev, class_current)
  if class_current == class_prev:
      #Checar si es la misma clase, si es la misma seguir haciendo append de las features
      for j in range(X_scaled.shape[1]):
        features_matrix_orig[j].append(X_scaled[i, j ])
        features_matrix_aug[j].append(X_scaled[(i + 1), j ])
  else:
      #sino generar los series y hacer el append al dataframe de la row
      row_dict_orig, row_dict_aug = {}, {}
      for k in range(X_scaled.shape[1]):
        row_dict_orig[k] = features_matrix_orig[k]
        row_dict_aug[k] = features_matrix_aug[k]
      row_dict_orig["class"] = class_current
      row_dict_aug["class"] = class_current
      #print(features_matrix_orig)
      df_transformer = df_transformer.append(row_dict_orig, ignore_index=True)
      df_transformer = df_transformer.append(row_dict_aug, ignore_index=True)
      for i in range(df.shape[1] - 1):
        features_matrix_orig.append([])
        features_matrix_aug.append([])
        columns.append(i)
      class_prev = class_current
  
  class_current = ""

#df_transformer.head(5)
df_transformer.to_excel("/content/gdrive/My Drive/your_directory/LIBRAS Data/"+video_str+"_fortransformer_output_17102022.xlsx")

In [ ]:
# import required module
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import os
import numpy as np
# assign directory
directory = '/content/gdrive/My Drive/your_directory/LIBRAS Data/'
 
# iterate over files in
# that directory
df_main = None
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and ('fortraining_output_07102022.xlsx' in f or 'fortraining_output_09092022' in f):
        print(f)
        df = pd.read_excel(f, index_col=0)
        if df_main is None:
          df_main = df.copy()
        else:
          df_main = df_main.append(df, ignore_index=True)

df_main.to_excel("/content/gdrive/My Drive/your_directory/LIBRAS Data/All_Vocab_28112022.xlsx")

ss = MinMaxScaler()
X = df_main.drop("label", axis=1) # Create a data with all columns except labels
X_scaled = ss.fit_transform(X)
pca = PCA(0.96)
pca.fit(X_scaled)
X_scaled = pca.transform(X_scaled)

features_matrix_orig = []
features_matrix_aug = []
columns = []
for i in range(X_scaled.shape[1]):
  features_matrix_orig.append([])
  features_matrix_aug.append([])
  columns.append(i)
class_prev = ""
class_current = ""
#create empty dataframe
df_transformer = pd.DataFrame(columns=columns)
# iterate through each row and select 
for i in range(0, len(df_main), 2) :
  class_current = df_main["label"][i]
  if class_prev == "":
    class_prev = class_current
  #print(class_prev, class_current)
  if class_current == class_prev:
      #Checar si es la misma clase, si es la misma seguir haciendo append de las features
      for j in range(X_scaled.shape[1]):
        features_matrix_orig[j].append(X_scaled[i, j ])
        features_matrix_aug[j].append(X_scaled[(i + 1), j ])
  else:
      #sino generar los series y hacer el append al dataframe de la row
      row_dict_orig, row_dict_aug = {}, {}
      for k in range(X_scaled.shape[1]):
        row_dict_orig[k] = features_matrix_orig[k]
        row_dict_aug[k] = features_matrix_aug[k]
      row_dict_orig["label"] = class_current
      row_dict_aug["label"] = class_current
      #print(features_matrix_orig)
      df_transformer = df_transformer.append(row_dict_orig, ignore_index=True)
      df_transformer = df_transformer.append(row_dict_aug, ignore_index=True)
      features_matrix_orig = []
      features_matrix_aug = []
      for i in range(X_scaled.shape[1]):
        features_matrix_orig.append([])
        features_matrix_aug.append([])
      class_prev = class_current
  
  class_current = ""

df_transformer.to_excel("/content/gdrive/My Drive/your_directory/LIBRAS Data/All_Vocab_Transformer_28112022.xlsx")